In [109]:
import os
import librosa
import music21
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

### DEFAULT_DRUM_TYPE_PITCHES

In [110]:
DEFAULT_DRUM_TYPE_PITCHES = [
    # kick drum
    [36, 35],
# 35: i.BassDrum,  # Acoustic Bass Drum
# 36: i.BassDrum,  # Bass Drum 1

    # snare drum
    [38, 27, 28, 31, 32, 33, 34, 37, 39, 40, 56, 65, 66, 75, 85],
# 37: i.SnareDrum,  # Side Stick
# 38: i.SnareDrum,  # Acoustic Snare
# 40: i.SnareDrum,  # Electric Snare
# 56: i.Cowbell,

    # closed hi-hat
    [42, 44, 54, 68, 69, 70, 71, 73, 78, 80, 22],
# 42: i.HiHatCymbal,  # Closed Hi Hat
# 44: i.HiHatCymbal,  # Pedal Hi-Hat
# 54: i.Tambourine,

    # open hi-hat
    [46, 67, 72, 74, 79, 81, 26],
# 46: i.HiHatCymbal,  # Open Hi-Hat
    # low tom
    [45, 29, 41, 43, 61, 64, 84],
# 41: i.TomTom,  # Low Floor Tom
# 43: i.TomTom,  # High Floor Tom
# 45: i.TomTom,  # Low Tom
# 61: i.BongoDrums,  # Low Bongo
# 64: i.CongaDrum,  # Low Conga

    # mid tom
    [48, 47, 60, 63, 77, 86, 87],
# 47: i.TomTom,  # Low-Mid Tom
# 48: i.TomTom,  # Hi-Mid Tom
# 60: i.BongoDrums,  # Hi Bongo
# 63: i.CongaDrum,  # Open Hi Conga

    # high tom
    [50, 30, 62, 76, 83],
# 50: i.TomTom,  # High Tom
# 62: i.CongaDrum,  # Mute Hi Conga


    # crash cymbal
    [49, 52, 55, 57, 58],
# 49: i.CrashCymbals,  # Crash Cymbal 1
# 57: i.CrashCymbals,  # Crash Cymbal 2
# 58: i.Vibraslap,

    # ride cymbal
    [51, 53, 59, 82],


    # Percussion
]

i = {
    # 65: i.Timbales,  # High Timbale
    # 66: i.Timbales,  # Low Timbale
    # 67: i.Agogo,  # High Agogo
    # 68: i.Agogo,  # Low Agogo
    # # 69: i.Cabasa,
    # 70: i.Maracas,
    # 71: i.Whistle,  # Short Whistle
    # 72: i.Whistle,  # Long Whistle
    # # 73: i.Short Guiro,
    # # 74: i.Long Guiro,
    # # 75: i.Claves,
    # 76: i.Woodblock,  # Hi Wood Block
    # 77: i.Woodblock,  # Low Wood Block
    # # 78: i.Mute Cuica,
    # # 79: i.Open Cuica,
    # 80: i.Triangle,  # Mute Triangle
    # 81: i.Triangle,  # Open Triangle
}

### midiPitchToInstrument

In [ ]:
def calculet_time(midi_part, tempo_sec):
    # midi.parts[i].flatten().notes
    peak_t = [-1]
    for parts in midi_part.parts:
        # top = parts.flatten().notes
        # print(len(midi_part.flatten()))
        for nt in parts.flatten().notes:
        # for nt in midi_part: 
            # 타악기.
            if isinstance(nt, percussion.PercussionChord) or isinstance(nt, note.Unpitched):
                if peak_t[-1] != nt.offset * tempo_sec:
                    peak_t.append(nt.offset * tempo_sec)


    return peak_t[1:]

In [137]:
from music21 import *

file = midi.MidiFile()
file.open("../../data/tmp_groove/122_funk_95_fill_4-4.mid")
file.read()
file.close()

# -- 95 bpm, 4/4 박자에서 4분 음표의 길이 0.631 => tempo_sec * offset = 현재 초
for e in file.tracks[0].events[:]:
    if e.type == midi.MetaEvents.SET_TEMPO:
        tempo_data = midi.translate.midiEventsToTempo(e)
        print(tempo_data)
        tempo_sec = tempo_data.secondsPerQuarter()
        bpm = tempo_data.getQuarterBPM()
        print(tempo_sec, bpm)
    elif e.type == midi.MetaEvents.TIME_SIGNATURE:
        print(midi.translate.midiEventsToTimeSignature(e))

<music21.tempo.MetronomeMark Quarter=95>
0.631578947368421 95.0
<music21.meter.TimeSignature 4/4>


In [ ]:
import matplotlib.pyplot as plt
from music21 import *

pm = midi.percussion.PercussionMapper()

def analyze_drum_midi(midi_file):
    midi_stream = converter.parse(midi_file)
    for element in midi_stream.flat.notes:
        if isinstance(element, note.Note):
            # For pitched notes
            print("Offset:", element.offset, "| Pitch:", element.pitch.midi, "| Duration:", element.duration.quarterLength)
        elif isinstance(element, chord.Chord):
            # For chords (potentially including PercussionChord)
            pitches = [p.midi for p in element.pitches]
            print("Offset:", element.offset, "| Pitches:", pitches, "| Duration:", element.duration.quarterLength)
        # elif isinstance(element, percussion.PercussionChord):
        elif isinstance(element, note.Unpitched):
            # if element.storedInstrument:
            print(element.storedInstrument.percMapPitch)
            # print(element.storedInstrument.instrumentName)
        elif isinstance(element, percussion.PercussionChord):
            allNotes = []
            for thisNote in element.notes:
                if isinstance(thisNote, note.Note):
                    allNotes.append(thisNote.nameWithOctave)
                elif isinstance(thisNote, note.Unpitched):
                    if thisNote.storedInstrument:
                        allNotes.append(thisNote.storedInstrument.percMapPitch)
                        # allNotes.append(str(thisNote.storedInstrument.thisNote.storedInstrument))
                    else:
                        allNotes.append(f'unpitched[{thisNote.displayName}]')


            print("percussion >> ", element.quarterLength)
analyze_drum_midi("../../data/tmp_groove/122_funk_95_fill_4-4.mid")

AttributeError: 'Score' object has no attribute 'tracks'

### 훈련

In [112]:
# Step 1: Audio Feature Extraction using librosa
max_pad_len = 174
def extract_audio_features(wav_file):
    audio, sample_rate = librosa.load(wav_file)
    # audio, sample_rate = librosa.load(file_name)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0,0), (0, pad_width)), mode='constant')
    
    return mfccs.T  # Transpose for sequence length as the first dimension

In [113]:
# Step 2: MIDI Information Extraction using music21
import matplotlib.pyplot as plt
from music21 import *

def extract_midi_information(midi_file):
    midi_stream = music21.converter.parse(midi_file)
    
    hits = []
    # 스트림 평탄화: 중첩된 스트림을 평탄화 한다.
    for element in midi_stream.flat.notes:
        if isinstance(element, note.Unpitched):# -- unpitched : 한 개 침
            hits.append({'offset' : float(element.offset),
                         'quarterLength': float(element.quarterLength),
                         'instrument': [element.storedInstrument.percMapPitch]})
        elif isinstance(element, percussion.PercussionChord):# -- percussion : 동시에 여러 개 침
            allNotes = []
            for thisNote in element.notes:
                if isinstance(thisNote, note.Note):
                    allNotes.append(thisNote.nameWithOctave)
                elif isinstance(thisNote, note.Unpitched):
                    if thisNote.storedInstrument:
                        allNotes.append(thisNote.storedInstrument.percMapPitch)
                    # else:
                    #     allNotes.append(f'unpitched[{thisNote.displayName}]')
            hits.append({'offset' : float(element.offset),
                'quarterLength': float(element.quarterLength),
                'instrument': allNotes}) ## -- element.storedInstrument
            
    return hits

In [114]:
# Step 3: Data Preparation
# Assuming you have a dataset with paired WAV and MIDI files
# and you have labeled MIDI information for training
def prepare_data(dataset_path):
    X = []  # Input: Audio Features
    y = []  # Output: MIDI Information (hit timing and drum type)

    for filename in os.listdir(dataset_path):
        if filename.endswith(".wav"):
            wav_path = os.path.join(dataset_path, filename)
            midi_path = os.path.join(dataset_path, filename.replace(".wav", ".mid"))

            # Extract features from WAV file
            audio_features = extract_audio_features(wav_path)
            X.append(audio_features)

            # Extract MIDI information
            midi_information = extract_midi_information(midi_path)
            y.append(midi_information)
    return X, y

In [115]:
# Step 4: Model Architecture
model = Sequential()
model.add(LSTM(128, input_shape=(174, 40), return_sequences=True))
model.add(Dense(2)) # 2 outputs: hit timing and drum type

### 훈련

In [116]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
X, y = prepare_data("../../data/tmp_groove/")
y_train = []
for y_data in y:
    tmp = []
    for data in y_data:
        labels_array = np.array([data['offset'], data['quarterLength'], data['instrument']])
        print(labels_array)
    y_train.append(tmp)

n_columns = 174    
n_row = 40       
n_channels = 1
n_classes = 4
X_train = np.array(X)
X_train = tf.reshape(X_train, [-1, n_row, n_columns, n_channels])
y_train = np.array(y_train)

<bound method Music21Object._getTimeSignatureForBeat of <music21.percussion.PercussionChord [Hi-Hat Cymbal Snare Drum]>>
<bound method Music21Object._getTimeSignatureForBeat of <music21.percussion.PercussionChord [Snare Drum Snare Drum]>>
<bound method Music21Object._getTimeSignatureForBeat of <music21.percussion.PercussionChord [Tom-Tom Tom-Tom]>>
<bound method Music21Object._getTimeSignatureForBeat of <music21.note.Unpitched object at 0x7f36cc533110>>
<bound method Music21Object._getTimeSignatureForBeat of <music21.note.Unpitched object at 0x7f36cc532d10>>
<bound method Music21Object._getTimeSignatureForBeat of <music21.percussion.PercussionChord [Snare Drum Snare Drum]>>
<bound method Music21Object._getTimeSignatureForBeat of <music21.percussion.PercussionChord [Tom-Tom Tom-Tom]>>
<bound method Music21Object._getTimeSignatureForBeat of <music21.note.Unpitched object at 0x7f36cc5a52d0>>
<bound method Music21Object._getTimeSignatureForBeat of <music21.note.Unpitched object at 0x7f36cc

/tmp/ipykernel_325/2785238704.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels_array = np.array([data['offset'], data['quarterLength'], data['instrument']])


In [117]:
# Step 5: Training
model.compile(optimizer='adam', loss='mean_squared_error')  # Adjust loss function as needed
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

ValueError: Training data contains 1 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.2`. Either provide more data, or a different value for the `validation_split` argument.

In [ ]:
def extract_notes(midi_part):
    parent_element = []
    ret = []
    print(f"extract: {len(midi_part.stream().flatten().notes)}")
    # print(len(midi_part.flatten()))
    for nt in midi_part.stream().flatten().notes:
    # for nt in midi_part: 
        # 타악기.
        if isinstance(nt, percussion.PercussionChord):
            allNotes = []
            for thisNote in nt.notes:
                if isinstance(thisNote, note.Note):
                    allNotes.append(thisNote.nameWithOctave)
                elif isinstance(thisNote, note.Unpitched): # 드럼의 경우 unpictched 인듯. <- 확인이 필요함.
                    if thisNote.storedInstrument:
                        allNotes.append(str(thisNote.storedInstrument.instrumentName))
                    else:
                        allNotes.append(f'unpitched[{thisNote.displayName}]')
            ret.append(max(0.0, 0 if allNotes[-1] not in temp else temp[allNotes[-1]]))
            parent_element.append(nt)
            
            print(allNotes, nt.duration.quarterLength, nt.offset, nt.seconds, nt.beat)
        elif isinstance(nt, note.Unpitched): # 드럼의 경우 unpictched 인듯. <- 확인이 필요함.
            if nt.storedInstrument:
                this_note = str(nt.storedInstrument.instrumentName)
            else:
                this_note = f'unpitched[{nt.displayName}]'
            ret.append(max(0.0, 0 if this_note not in temp else temp[this_note]))
            parent_element.append(nt)
            print(this_note, nt.duration.quarterLength, nt.offset, nt.seconds, nt.beat)
        else:
            print(nt)
    
    return ret, parent_element